In [2]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
from PIL import Image

In [9]:
import os
import pathlib
import csv

LabelEncoder : Encode labels with value between 0 and n_classes-1
- can be used to normalize labels.
- It can also be used to transform non-numerical labels (as long as they are hashable and comparable) to numerical labels.

StandardScaler : Standardize features by removing the mean and scaling to unit variance


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import keras

### Dataset

GTZAN genre collection dataset. 
Consists of 10 genres, each with 1000 songs

The data contains the following features extracted from the spectrogram:
- MFCC (20)
- Spectral centroid
- zero crossing rate
- chroma frequencies 
- spectral roll-off

In [18]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00081.au,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,-46.809993,...,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,blues.00022.au,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,3.289490,...,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,blues.00031.au,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843071,18.620984,...,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,blues.00012.au,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,-15.438986,...,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,blues.00056.au,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,-12.517677,...,7.457218,-10.470444,-2.360483,-6.783624,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


In [19]:
data.shape

(1000, 28)

In [20]:
#dropping unnecessary columns
data = data.drop(['filename'], axis=1)

### Encoding labels

In [22]:
genre_list = data.iloc[:,-1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [31]:
#each row of the data assigned a label
#y

### Scaling feature columns

StandardScaler => features with mean=0 and variance=1

(Xi - Xmean) / (sd of feature)

In [26]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

In [37]:
X[0]

array([ 0.01965206,  1.78719576, -0.11862895, -0.54355152, -0.23862667,
        0.56525258,  1.17331702,  0.24849119, -1.74726783,  0.27850822,
       -1.66309873,  0.86035453, -1.39040065,  0.49083598, -0.97274341,
        0.57128411, -1.35173991,  1.46932942, -1.46316182,  1.15463556,
       -0.4743645 , -0.17868797, -0.62926671,  0.11859349, -1.3965024 ,
        1.0464641 ])

### Dividing data into training and testing data

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
print(len(y_train))
print(len(y_test))

800
200


In [38]:
X_train[10]

array([-1.36318755, -1.51161047, -1.51425623, -1.02720198, -1.63790319,
       -1.36993657, -2.68598489,  1.40096779,  1.85692119, -1.33634611,
        0.21864003, -1.4301045 , -0.13578662, -1.7423326 ,  0.12933912,
       -1.53682115,  0.47505071, -0.78658234,  0.58190486, -0.83131746,
        0.01230482, -1.01891476,  0.2759037 , -0.74811827,  0.7863191 ,
        0.98091824])

In [42]:
X_train.shape

(800, 26)

### Classification with Keras

#### Building the network

In [39]:
from keras import models
from keras import layers

Regular densely-connected NN layer
- units: Positive integer, dimensionality of the output space.
- activation: Activation function to use (see activations). Default - no activation is applied (ie. "linear" activation: a(x) = x).
- input shape : nD tensor. The most common situation would be a 2D input with shape (batch_size, input_dim).

In [43]:
model = models.Sequential() #linear stack of layers

model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [44]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [45]:
history = model.fit(X_train, y_train, epochs=20, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
800/800 [==============================] - 0s 587us/step - loss: 2.1983 - acc: 0.2512
Epoch 2/20
800/800 [==============================] - 0s 33us/step - loss: 1.8828 - acc: 0.3900
Epoch 3/20
800/800 [==============================] - 0s 40us/step - loss: 1.6489 - acc: 0.4100
Epoch 4/20
800/800 [==============================] - 0s 39us/step - loss: 1.4635 - acc: 0.4913
Epoch 5/20
800/800 [==============================] - 0s 38us/step - loss: 1.3332 - acc: 0.5275
Epoch 6/20
800/800 [==============================] - 0s 38us/step - loss: 1.2290 - acc: 0.5800
Epoch 7/20
800/800 [==============================] - 0s 35us/step - loss: 1.1349 - acc: 0.6225
Epoch 8/20
800/800 [==============================] - 0s 42us/step - loss: 1.0539 - acc: 0.6537
Epoch 9/20
800/800 [==============================] - 0s 40us/step - loss: 0.9872 - acc: 0.6950
Epoch 10/20
800/800 [==============================] - 0s 40us/step - loss: 0.9323 - ac

In [46]:
test_loss, test_acc = model.evaluate(X_test, y_test)

200/200 [==============================] - 0s 293us/step


In [47]:
print('Test Accuracy: ',test_acc)

Test Accuracy:  0.685


(test acc < training acc) => Overfitting

### Using validation

The validation is done along with the training during each epoch, except the errors from the validation set classification isn't propagated back for weights and bias adjustment. 


In [61]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [62]:
len(partial_y_train)

600

In [63]:
model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 1s 1ms/step - loss: 2.3028 - acc: 0.1317 - val_loss: 2.1683 - val_acc: 0.2350
Epoch 2/30
600/600 [==============================] - 0s 54us/step - loss: 2.1393 - acc: 0.2833 - val_loss: 2.0713 - val_acc: 0.2550
Epoch 3/30
600/600 [==============================] - 0s 81us/step - loss: 2.0205 - acc: 0.3100 - val_loss: 1.9846 - val_acc: 0.2750
Epoch 4/30
600/600 [==============================] - 0s 54us/step - loss: 1.9007 - acc: 0.3333 - val_loss: 1.9067 - val_acc: 0.3050
Epoch 5/30
600/600 [==============================] - 0s 53us/step - loss: 1.7838 - acc: 0.3767 - val_loss: 1.8253 - val_acc: 0.3150
Epoch 6/30
600/600 [==============================] - 0s 45us/step - loss: 1.6726 - acc: 0.3967 - val_loss: 1.7439 - val_acc: 0.3550
Epoch 7/30
600/600 [==============================] - 0s 45us/step - loss: 1.5731 - acc: 0.4433 - val_loss: 1.6621 - val_acc: 0.4050
Epoch 8/30
600/600 [====

In [66]:
results = model.evaluate(X_test, y_test)
results #loss value and metrics value

200/200 [==============================] - 0s 74us/step


[1.0222037315368653, 0.64]